<a href="https://colab.research.google.com/github/dylanJzch/Predict-Credit-Card-Default/blob/main/predict_credit_card_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# **Data preprocessing**


In [27]:
from google.colab import drive
drive.mount('/content/drive')
df_raw = pd.read_excel("/content/drive/MyDrive/BDML/default_of_credit_card_clients.xls")

df = df_raw.iloc[1:,1:] #remove the first row and first column
df.columns = df_raw.iloc[0].values[1:]
print(df.shape)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(30000, 24)


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# **Create train and test for the RNN and LSTM models**


*random split the whole data set*

In [28]:
# 20% of the data will be used for the test set, and the remaining 80% will be used for the training set. 
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [29]:
## PARTITION by age
female_over_30 = df[(df['SEX'] == 2) & (df['AGE'] > 30)] ## Female over 30
female_under_30 = df[(df['SEX'] == 2) & (df['AGE'] < 30)] ## Female under 30

male_over_30 = df[(df['SEX'] == 1) & (df['AGE'] > 30)] ## Male over 30
male_under_30 = df[(df['SEX'] == 1) & (df['AGE'] < 30)] ## Male under 30
##-------------------------------------------------------------------------------------------##
## Partition by education
## Female over 30: 
female_over_30_grad = female_over_30[female_over_30['EDUCATION'] == 1]  # graduate school
female_over_30_uni = female_over_30[female_over_30['EDUCATION'] == 2]  # university
female_over_30_highschool = female_over_30[female_over_30['EDUCATION'] == 3]  # high school
female_over_30_others = female_over_30[female_over_30['EDUCATION'] == 4]  # others
## Female under 30: 
female_under_30_grad = female_under_30[female_under_30['EDUCATION'] == 1]  # graduate school
female_under_30_uni = female_under_30[female_under_30['EDUCATION'] == 2]  # university
female_under_30_highschool = female_under_30[female_under_30['EDUCATION'] == 3]  # high school
female_under_30_others = female_under_30[female_under_30['EDUCATION'] == 4]  # others
## Male over 30: 
male_over_30_grad = male_over_30[male_over_30['EDUCATION'] == 1]  # graduate school
male_over_30_uni = male_over_30[male_over_30['EDUCATION'] == 2]  # university
male_over_30_highschool = male_over_30[male_over_30['EDUCATION'] == 3]  # high school
male_over_30_others = male_over_30[male_over_30['EDUCATION'] == 4]  # others
## Male under 30: 
male_under_30_grad = male_under_30[male_under_30['EDUCATION'] == 1]  # graduate school
male_under_30_uni = male_under_30[male_under_30['EDUCATION'] == 2]  # university
male_under_30_highschool = male_under_30[male_under_30['EDUCATION'] == 3]  # high school
male_under_30_others = male_under_30[male_under_30['EDUCATION'] == 4]  # others 
male_married = df[(df['SEX'] == 1) & (df['MARRIAGE'] == 1)]
female_married = df[(df['SEX'] == 2) & (df['MARRIAGE'] == 1)]

In [30]:
# Define the training set
df_train = male_married
# Define the list of other partitions
#other_partitions = male_under_30_grad
df_test = female_married
# Concatenate other partitions to form the test set
#df_test = pd.concat(other_partitions)

In [31]:
# Preparing data for the feedforward network
X_train = df_train.drop('default payment next month', axis=1)
y_train = df_train['default payment next month']

In [32]:
# reShape for LSTM
X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))

# **3 Time Series**
### *1: time_series_payment*
### *2: time_series_bill_amt*
### *3: time_series_pay_amt*

In [33]:
df_train.shape

(5190, 24)

In [34]:
time_series_payment = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', ]
array1 = df_train[time_series_payment].values.astype(float)
print(array1.shape) # 24000 users with 6 features

(5190, 6)


In [35]:
time_series_bill_amt = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',  ]
array2 = df_train[time_series_bill_amt].values.astype(float)
print(array2.shape) # 24000 users with 6 features

(5190, 6)


In [36]:
time_series_pay_amt = ['PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
array3 = df_train[time_series_pay_amt].values.astype(float)
print(array3.shape) # 24000 users with 6 features

(5190, 6)


In [37]:
# Stack arrays along new axis
X_train = np.stack([array1, array2, array3], axis=1)

print(X_train.shape) 

(5190, 3, 6)


In [38]:
y_train = df_train['default payment next month'].astype(int).values
print(y_train.shape)

(5190,)


# **Preparing data for the test set**

*Get X_test, y_test*


In [39]:
X_test = df_test.drop('default payment next month', axis=1)
y_test = df_test['default payment next month']

In [40]:
# Create the 3 arrays for the 3 time-series
time_series_payment = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', ]
array1_test = df_test[time_series_payment].values.astype(float)

time_series_bill_amt = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',  ]
array2_test = df_test[time_series_bill_amt].values.astype(float)

time_series_pay_amt = ['PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
array3_test = df_test[time_series_pay_amt].values.astype(float)

# Stack arrays along new axis
X_test = np.stack([array1_test, array2_test, array3_test], axis=1)

print(f'X test:',X_test.shape)
print(f'Y test:',y_test.shape)

X test: (8469, 3, 6)
Y test: (8469,)


# **LSTM Model**


In [41]:
# Define model
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(3, 6)))
model.add(Dropout(0.3))  # 20% dropout
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(Dropout(0.3))  # 20% dropout
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1))

# Compile the LSTM model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# Summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 3, 128)            69120     
                                                                 
 dropout_3 (Dropout)         (None, 3, 128)            0         
                                                                 
 lstm_4 (LSTM)               (None, 3, 64)             49408     
                                                                 
 dropout_4 (Dropout)         (None, 3, 64)             0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                

### Train the model:

In [45]:
# Train the feedforward network
model.fit(X_train, y_train, epochs=35, batch_size=128, 
          validation_split=0.2, callbacks=[es])

# Evaluate the feedforward network


Epoch 1/35
33/33 [==============================] - 5s 73ms/step - loss: 4.0697 - accuracy: 0.7211 - val_loss: 3.6636 - val_accuracy: 0.7390
Epoch 2/35
33/33 [==============================] - 2s 46ms/step - loss: 4.0679 - accuracy: 0.7223 - val_loss: 3.6902 - val_accuracy: 0.7372
Epoch 3/35
33/33 [==============================] - 1s 42ms/step - loss: 4.0483 - accuracy: 0.7213 - val_loss: 3.7014 - val_accuracy: 0.7381
Epoch 4/35
33/33 [==============================] - 1s 26ms/step - loss: 3.9708 - accuracy: 0.7254 - val_loss: 3.6431 - val_accuracy: 0.7416
Epoch 5/35
33/33 [==============================] - 1s 27ms/step - loss: 4.0573 - accuracy: 0.7228 - val_loss: 3.6421 - val_accuracy: 0.7416
Epoch 6/35
33/33 [==============================] - 1s 27ms/step - loss: 3.9503 - accuracy: 0.7312 - val_loss: 3.6420 - val_accuracy: 0.7416
Epoch 7/35
33/33 [==============================] - 1s 26ms/step - loss: 4.0407 - accuracy: 0.7237 - val_loss: 3.6569 - val_accuracy: 0.7407
Epoch 8/35
33

# Evaluate the model

In [46]:
# Evaluate on test data
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

loss, accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


265/265 [==============================] - 2s 6ms/step - loss: 3.2569 - accuracy: 0.7799
Test Loss: 3.2569286823272705
Test Accuracy: 0.7799031734466553
